# Mixtures of Logit Swissmetro: 1000 draws

Have two randoms coefficients (cost + time) and heterogeneity in the ASCs



Options for the few_draws.toml:
- Second derivative set to 0
- Number of draws: 1000 (for now)



In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable, bioDraws, MonteCarlo, log, Power, exp, Derive, RandomVariable, PanelLikelihoodTrajectory
from biogeme.tools import TemporaryFile

Preparing the data

In [2]:
url = "http://transp-or.epfl.ch/data/swissmetro.dat"

# Read the data into a DataFrame
df = pd.read_csv(url, sep='\t')
database_swissmetro = db.Database('swissmetro', df)

#Definition of the variables:

GROUP = Variable('GROUP')
SURVEY = Variable('SURVEY')
SP = Variable('SP')
ID = Variable('ID')
PURPOSE = Variable('PURPOSE')
FIRST = Variable('FIRST')
TICKET = Variable('TICKET')
WHO = Variable('WHO')
LUGGAGE = Variable('LUGGAGE')
AGE = Variable('AGE')
MALE = Variable('MALE')
INCOME = Variable('INCOME')
GA = Variable('GA')
ORIGIN = Variable('ORIGIN')
DEST = Variable('DEST')
TRAIN_AV = Variable('TRAIN_AV')
CAR_AV = Variable('CAR_AV')
SM_AV = Variable('SM_AV')
TRAIN_TT = Variable('TRAIN_TT')
TRAIN_CO = Variable('TRAIN_CO')
TRAIN_HE = Variable('TRAIN_HE')
SM_TT = Variable('SM_TT')
SM_CO = Variable('SM_CO')
SM_HE = Variable('SM_HE')
SM_SEATS = Variable('SM_SEATS')
CAR_TT = Variable('CAR_TT')
CAR_CO = Variable('CAR_CO')
CHOICE = Variable('CHOICE')

#We estimate the parameters of the model using all observations in the data set associated with work trips. 
#Observations such that the dependent variable CHOICE is 0 are also removed.

exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
database_swissmetro.remove(exclude)

#Definition of new variables:
SM_COST = database_swissmetro.DefineVariable('SM_COST', SM_CO * (GA == 0))
TRAIN_COST = database_swissmetro.DefineVariable('TRAIN_COST', TRAIN_CO * (GA == 0))
CAR_AV_SP = database_swissmetro.DefineVariable('CAR_AV_SP', CAR_AV * (SP != 0))
TRAIN_AV_SP = database_swissmetro.DefineVariable('TRAIN_AV_SP', TRAIN_AV * (SP != 0))
TRAIN_TT_SCALED = database_swissmetro.DefineVariable('TRAIN_TT_SCALED', TRAIN_TT / 100)
TRAIN_COST_SCALED = database_swissmetro.DefineVariable('TRAIN_COST_SCALED', TRAIN_COST / 100)
SM_TT_SCALED = database_swissmetro.DefineVariable('SM_TT_SCALED', SM_TT / 100)
SM_COST_SCALED = database_swissmetro.DefineVariable('SM_COST_SCALED', SM_COST / 100)
CAR_TT_SCALED = database_swissmetro.DefineVariable('CAR_TT_SCALED', CAR_TT / 100)
CAR_CO_SCALED = database_swissmetro.DefineVariable('CAR_CO_SCALED', CAR_CO / 100)

database_swissmetro.panel('ID') 



Defining Model Parameters

In [3]:
ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_TRAIN = Beta('ASC_TRAIN', 0, None, None, 0)
ASC_SM = Beta('ASC_SM', 0, None, None, 1) #Setting it to 0, no estimation

Defining TWandom parameter, with Halton Draw, for Monte-Carlo Simulation

In [4]:
B_TIME = Beta('B_TIME', 0, None, None, 0)


B_COST = Beta('B_COST', 0, None, None, 0)
B_COST_S = Beta('B_COST_S', 1, None, None, 0) 
B_COST_RND = B_COST + B_COST_S * bioDraws('B_COST_RND', 'NORMAL_HALTON3')


B_TIME_S = Beta('B_TIME_S', 1, None, None, 0) 
B_TIME_RND = B_TIME + B_TIME_S * bioDraws('B_TIME_RND', 'NORMAL_HALTON3')




Defining the Model

In [5]:
V1 = ASC_TRAIN + B_TIME_RND * TRAIN_TT_SCALED + B_COST_RND * TRAIN_COST_SCALED 
V2 = ASC_SM + B_TIME_RND * SM_TT_SCALED + B_COST_RND * SM_COST_SCALED 
V3 = ASC_CAR + B_TIME_RND * CAR_TT_SCALED + B_COST_RND * CAR_CO_SCALED

V = {1: V1, 2: V2, 3: V3}

av = {1: TRAIN_AV_SP, 2: SM_AV, 3: CAR_AV_SP}

Estimating the Model

In [6]:
prob = models.logit(V, av, CHOICE)

condprodIndiv = PanelLikelihoodTrajectory(prob) #will use panel data in this case

logprob = log(MonteCarlo(condprodIndiv))


USER_NOTES = (
    'Example of a mixture of logit models with three alternatives, '
    'approximated using Monte-Carlo integration. Two randoms coefficients.'
)

the_biogeme = bio.BIOGEME(
    database_swissmetro, logprob, userNotes=USER_NOTES, parameter_file='few_draws_2.toml'
)
the_biogeme.modelName = 'swissmetro_Halton_Mixture_2_COST_TIME - newton'

results = the_biogeme.estimate()


In [7]:
# Retrieve the general statistics from the results
general_stats = results.getGeneralStatistics()
print(results.printGeneralStatistics())

Number of estimated parameters:	6
Sample size:	752
Observations:	6768
Excluded observations:	3960
Init log likelihood:	-5611.954
Final log likelihood:	-4266.805
Likelihood ratio test for the init. model:	2690.297
Rho-square for the init. model:	0.24
Rho-square-bar for the init. model:	0.239
Akaike Information Criterion:	8545.611
Bayesian Information Criterion:	8573.347
Final gradient norm:	4.8078E-03
Number of draws:	100
Draws generation time:	0:00:00.096737
Types of draws:	['B_COST_RND: NORMAL_HALTON3', 'B_TIME_RND: NORMAL_HALTON3']
Nbr of threads:	8



In [8]:
pandas_results = results.getEstimatedParameters()
pandas_results

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR,0.184535,0.112411,1.641604,1.006721e-01
ASC_TRAIN,-0.528831,0.132597,-3.988266,6.655796e-05
B_COST,-2.469140,0.161156,-15.321423,0.000000e+00
B_COST_S,1.732096,0.230596,7.511386,5.839773e-14
B_TIME,-3.509216,0.194373,-18.054026,0.000000e+00
B_TIME_S,3.998340,0.218875,18.267675,0.000000e+00
